# Set up

In [4415]:
from worde4mde import load_embeddings
sgram_mde = load_embeddings('sgram-mde')

In [4416]:
word = 'ecore'
sgram_mde[word]

array([ 0.14674647,  0.42704162,  0.17717203,  0.05179158,  0.38020504,
       -0.00091264,  0.02094162,  0.00924711, -0.13806985,  0.1403339 ,
       -0.0953936 , -0.03681302,  0.38122246, -0.03746711, -0.17595871,
       -0.02031669,  0.23675247,  0.27676958, -0.07487358, -0.1987225 ,
        0.02015282,  0.05147164, -0.223504  ,  0.00699752, -0.06145973,
        0.00920826,  0.01364714,  0.10218145, -0.25330105, -0.14080754,
        0.2541444 ,  0.4577827 , -0.16825216, -0.08050938,  0.11404785,
       -0.38822263, -0.06395938, -0.4724689 , -0.00111705,  0.27869806,
        0.28032073,  0.02976846, -0.01675524,  0.09774698,  0.18298927,
       -0.24398401, -0.3238438 ,  0.01912955,  0.11861664,  0.08640955,
        0.18799016, -0.06891091, -0.03893414, -0.04862301,  0.02151174,
       -0.39205688, -0.23195332, -0.09532862, -0.01324714, -0.24555108,
        0.09372689, -0.21971853, -0.26171875, -0.3605202 , -0.07306654,
       -0.16288763,  0.12160088,  0.11416717,  0.17378834, -0.04

In [4417]:
# import pickle

import ast
import pandas as pd
import numpy as np

from worde4mde import load_embeddings
import re

import os
import openai
from openai import OpenAI
import json

import networkx as nx
import sys


In [4418]:
from dotenv import load_dotenv

# %%

load_dotenv()

openai_key = os.getenv('openai_key')
os.environ['OPENAI_API_KEY'] = openai_key

## Input

In [4419]:
##### update here to change input file directory ######
##### update here to change input file directory ######
##### update here to change input file directory ######
##### update here to change input file directory ######

instructor_dir = "results/"

# 01_LabTracker
# 02_CelO
# 03_TSS
# 04_SHAS
# 05_OTS
# 06_Block
# 07_Tile
# 08_HBMS
problem_id = "08_HBMS"

# location of reference domain model is stored. It is in .txt format
instructor_input = instructor_dir + problem_id + ".txt"

# gpt-4o-mini
# gpt-4
# gpt-4o
llm_id = "gpt-4"

# yujing
# sherpa
method_id = "yujing"

student_dir = instructor_dir + llm_id + '/' + method_id + '/' + problem_id + '/'
if not os.path.exists(student_dir):
    # Create the directory
    os.makedirs(student_dir)
    
# location of generated domain model is saved. The file should be in .txt    
student_input = student_dir + problem_id + ".txt"
student_input
student_out_dir = student_dir + "result/"
student_out_dir

'results/gpt-4/yujing/08_HBMS/result/'

In [4420]:
# First of all, you need to load the embeddings (currently supported: 'sgram-mde' and 'glove-mde')
sgram_mde = load_embeddings('sgram-mde')
# glove = load_embeddings('glove-mde')
# sgram_mde["id"]

In [4421]:
# word = 'id'
# sgram_mde.most_similar(positive=[word])

In [4422]:
def inEmbedding(word, embedding):
  try:
    return True, embedding[word]
  except:
    return False, None

In [4423]:

def splitCamelCase(word):
  splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', word)).split()
  return splitted

splitCamelCase("device ID")

['device', 'ID']

In [4424]:
splitCamelCase("CommandStatus")

['Command', 'Status']

In [4425]:
# client = OpenAI()

# def run_llm(prompt, model ="gpt-3.5-turbo"):
#   response = client.chat.completions.create(
#     model=model,
#     messages=[
#       {"role": "user", "content": prompt},

#     ]
#   )
#   return response.choices[0].message.content

In [4426]:
client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [4427]:
def get_all_info(class_index, class_nodes, list_of_classes, list_edges, need_edge = True):
  result = ''
  result +=  list_of_classes[class_index] + "\n"
  node = class_nodes[class_index]
  if "abstract" in node:
      node = node.replace("abstract", "").strip()
  if "Abstract" in node:
      node = node.replace("Abstract", "").strip()
  if need_edge:
    for edge in list_edges:
      element = [i.strip() for i in edge.split()]
      if node in element:
        result += edge + "\n"

  return  result

In [4428]:
def match_classes(raw_1, raw_2, dict_attr, thresh = 0.5, verbose = False):
  # raw_1: nodes from reference solution
  # raw_2: nodes from student solution
  # dict_attr: dictionary for embeddings
  # thresh for subst cost
  # map class lists raw_1 and raw_2
  print(dict_attr)
  def node_subst_cost_attr(node1, node2):
    # threshod as 0.45
    if dict_attr[node1["name"]][node2["name"]] < thresh:
      return 3
    else:
      return 1-dict_attr[node1["name"]][node2["name"]]
    


  G_att_1 = nx.Graph()
  G_att_2 = nx.Graph()
  for node in raw_1:
    G_att_1.add_node(node, name = node)

  for node in raw_2:
    G_att_2.add_node(node, name = node)

  for v in nx.optimize_edit_paths(G_att_1, G_att_2, node_subst_cost=node_subst_cost_attr, edge_match=None, timeout = 20):
      minv = v
  # minv

  return minv

# Group

In [4429]:
class Grader():
  def __init__(self):
    self.ref = None
    self.stu = None

In [4430]:
grader = Grader()

## Class

### Define metadata

In [4431]:
class ModelInfo():

  def __init(self):
    self.cls_atr = {}
    self.cls_name = []
    self.raw_dsl = []
    self.rel = None

In [4432]:
ref_cls = ModelInfo()
stu_cls = ModelInfo()

In [4433]:
grader.ref = ref_cls
grader.stu = stu_cls

In [4434]:
with open(instructor_input, 'r') as file:
    # Read the entire content of the file
    instructor_all = file.read()

with open(student_input, 'r') as file:
    # Read the entire content of the file
    student_all = file.read()

In [4435]:
print(instructor_all)

Enumerations:
RoomType(SINGLE, DOUBLE, TWIN)
BookingStatus(FINALIZED, CONFIRMED, CANCELLED_BY_HBMS, PRELIMINARY, CANCELLED_BY_TRAVELLER, CANCELLED_BY_HOTEL)
PaymentType (PREPAID, PAID_AT_HOTEL)

Classes:
HotelBookingManager()
HotelChain()

Traveller (string name, int reliabilityRating)

BillingInformation(string companyName, string address)
TravelPreference(string preference, string value)
Hotel (string city, string country, string area)
abstract Offer()
SpecialOffer()
RegularOffer()
TravelSearch()
SearchInfo()
RoomAvailability()
abstract TripInfo(int numberOfRooms, RoomType roomType, date arrivalDate, date departureDate, float budgetPerNight)
BookingInfo()
Booking (int bookingId, date cancellationDeadline, string creditCardNumber, BookingStatus bookingStatus, PaymentType paymentType, date confirmationDate, float paidAmount)


Relationships:



In [4436]:
print(student_all)

Enumerations:
RoomType(Single, Double, Twin)
BookingStatus(Preliminary, Regular, Finalized, Confirmed, Cancelled)
PaymentType(PrePaid, PaidAtHotel)
HotelRating(OneStar, TwoStar, ThreeStar, FourStar, FiveStar)

Classes:

HotelBookingManagementSystem(string city, date dateOfArrival, date dateOfDeparture, int numberOfRooms, string typeOfRooms, int hotelRating, float budget, string travelPeriod, string bookingStatus, string bookingIdentifier)
BusinessTraveller(string name, BillingInformation billingInformation, TravelPreference travelPreference, ReliabilityRating reliabilityRating)

BillingInformation(string companyName, string address)
TravelPreference(boolean breakfastIncluded, boolean freeWifi, boolean frontDesk)
Hotel(string name, string address, string city, string hotelChain, RoomAvailability roomAvailability)

SpecialOffer(float price, string cityArea, int hotelRating, string keyPreferences)



RoomAvailability(string typeOfRoom, boolean isAvailable)


RegularBooking(string bookingI

In [4437]:
shas_sol = instructor_all.split("Relationships:")[0]
stu_sol_g =  student_all.split("Relationships:")[0] # student solution

In [4438]:
ref_classes_raw = []
tmp = shas_sol.strip().splitlines()
ref_classes_raw = [i.strip() for i in tmp if len(i) > 0]
ref_classes_raw

reference_class = []
tmp = shas_sol.strip().splitlines()
reference_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]

In [4439]:
stu_classes_raw = []
tmp = stu_sol_g.strip().splitlines()
stu_classes_raw = [i.strip() for i in tmp if len(i) > 0]
stu_classes_raw

tmp = stu_sol_g.strip().splitlines()
stu_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]
stu_class

['Enumerations:',
 'RoomType',
 'BookingStatus',
 'PaymentType',
 'HotelRating',
 'Classes:',
 'HotelBookingManagementSystem',
 'BusinessTraveller',
 'BillingInformation',
 'TravelPreference',
 'Hotel',
 'SpecialOffer',
 'RoomAvailability',
 'RegularBooking',
 'AccommodationDeal',
 'abstract BookingStatus',
 'PreliminaryBooking',
 'FinalizedBooking',
 'ConfirmedBooking',
 'CancelledBooking',
 'CreditCardInformation']

In [4440]:
ref_attributes = {}
enum_index = ref_classes_raw.index("Enumerations:")
regular_index = ref_classes_raw.index("Classes:")
for cla, dsl in zip(reference_class, ref_classes_raw):
  if dsl == "Enumerations:" or dsl == "Classes:":
    continue
  else:
    index = ref_classes_raw.index(dsl)
    if index > regular_index:
      class_type = "regular"
      if "abstract" in dsl:
        class_type = "abstract"
    else:
      class_type = "enum"
    ref_attributes[cla] = {"score": 0, "type":class_type, "dsl":dsl, "counterpart": None, "attributes": {}}

    attributes = dsl.split("(")[1][:-1].split(",")
    for attr in attributes:
      attr = attr.strip()
      if len(attr) > 0:
        ref_attributes[cla]["attributes"][attr] = {"score":0, "counterpart": None}

In [4441]:
reference_class.remove("Enumerations:")
reference_class.remove("Classes:")

ref_classes_raw.remove("Enumerations:")
ref_classes_raw.remove("Classes:")

In [4442]:
ref_cls.cls_name = reference_class
ref_cls.raw_dsl = ref_classes_raw
ref_cls.cls_atr = ref_attributes

In [4443]:
stu_attributes = {}
enum_index = stu_classes_raw.index("Enumerations:")
regular_index = stu_classes_raw.index("Classes:")

for cla, dsl in zip(stu_class, stu_classes_raw):
  if dsl == "Enumerations:" or dsl == "Classes:":
    continue
  else:
    index = stu_classes_raw.index(dsl)
    if index > regular_index:
      class_type = "regular"
      if "abstract" in dsl:
        class_type = "abstract"
    else:
      class_type = "enum"

    stu_attributes[cla] = {"score": 0, "type":class_type, "dsl":dsl, "counterpart": None, "attributes": {}}

    print(dsl)
    attributes = dsl.split("(")[1][:-1].split(",")
    for attr in attributes:
      attr = attr.strip()
      if len(attr) > 0:
        stu_attributes[cla]["attributes"][attr] = {"score":0, "counterpart": None}

RoomType(Single, Double, Twin)
BookingStatus(Preliminary, Regular, Finalized, Confirmed, Cancelled)
PaymentType(PrePaid, PaidAtHotel)
HotelRating(OneStar, TwoStar, ThreeStar, FourStar, FiveStar)
HotelBookingManagementSystem(string city, date dateOfArrival, date dateOfDeparture, int numberOfRooms, string typeOfRooms, int hotelRating, float budget, string travelPeriod, string bookingStatus, string bookingIdentifier)
BusinessTraveller(string name, BillingInformation billingInformation, TravelPreference travelPreference, ReliabilityRating reliabilityRating)
BillingInformation(string companyName, string address)
TravelPreference(boolean breakfastIncluded, boolean freeWifi, boolean frontDesk)
Hotel(string name, string address, string city, string hotelChain, RoomAvailability roomAvailability)
SpecialOffer(float price, string cityArea, int hotelRating, string keyPreferences)
RoomAvailability(string typeOfRoom, boolean isAvailable)
RegularBooking(string bookingIdentifier)
AccommodationDeal(str

In [4444]:
stu_class.remove("Enumerations:")
stu_class.remove("Classes:")

stu_classes_raw.remove("Enumerations:")
stu_classes_raw.remove("Classes:")

In [4445]:
stu_cls.cls_name = stu_class
stu_cls.raw_dsl = stu_classes_raw
stu_cls.cls_atr = stu_attributes

## Edges

### reference and student solutions

In [4446]:
shas_edges = instructor_all.split("Relationships:")[1]
shas_G2_edges =  student_all.split("Relationships:")[1] # student solution

In [4447]:
shas_edges

'\n'

In [4448]:
shas_edges

'\n'

In [4449]:
shas_edges = shas_edges.replace("0..*","*")

In [4450]:
shas_G2_edges = shas_G2_edges.replace("0..*","*")

In [4451]:
tmp = shas_edges.strip().splitlines()
ref_edges = [i.strip() for i in tmp if len(i) > 0]

for i in ref_edges:
  length = len(i.split())
  if length == 5 or length == 3:
    pass
  else:
    print("Reference Length error:", i)

tmp = shas_G2_edges.strip().splitlines()
stu_edges = [i.strip() for i in tmp if len(i) > 0]

for i in stu_edges:
  length = len(i.split())
  if length == 5 or length == 3:
    pass
  else:
    print("Student Length error:", i)



In [4452]:
class Relationship:
  def __init__(self):
    self.rels = [] # list of dict{}
    self.raw_dsl = []
  # find first non-matched or matched reltionshipi given dsl
  def find_relation(self, dsl, matched = False):
    for index, i in enumerate(self.rels):
      if i["dsl"] == dsl and i["counterpart"] is None:
        return index, i

    return None

In [4453]:
ref_edges_obj = Relationship()
stu_edges_obj = Relationship()


ref_edges_dict = []
for e in ref_edges:
  ref_edges_dict.append({"dsl":e,"score":0, "counterpart": None})

ref_edges_obj.rels = ref_edges_dict
ref_edges_obj.raw_dsl = ref_edges

stu_edges_dict = []
for e in stu_edges:
  stu_edges_dict.append({"dsl":e,"score":0, "counterpart": None})

stu_edges_obj.rels = stu_edges_dict
stu_edges_obj.raw_dsl = stu_edges

In [4454]:
# link to classInfo object
ref_cls.rel = ref_edges_obj
stu_cls.rel = stu_edges_obj

In [4455]:
edges = [ref_edges_obj, stu_edges_obj]

## Getting embedding

In [4456]:
def removeChars(text, chars):
    for c in chars:
        text = text.replace(c,"")
    return text


In [4457]:
def get_mde_embedding(text, embedding):
  # >>> get_mde_embedding("WhatDevice", sgram_mde)
  words = splitCamelCase(removeChars(text, ["(",")",","]))
  lowercase_list = [s.lower() for s in words]
  counter = 0
  emb = np.zeros(300)
  for w in lowercase_list:
    try:
      emb += embedding[w]
      counter += 1
    except:
      # if failed in emb the complete word, embed the token
      tmp = np.zeros(300)
      for char in list(w):

        tmp += embedding[char]
      emb += (tmp / len(w))
      counter += 1
  return emb / counter

In [4458]:
text = "AutomationStatus (Created, Edited, Activated, Deactivated)"
words = splitCamelCase(removeChars(text, ["(",")",","]))
words

['Automation', 'Status', 'Created', 'Edited', 'Activated', 'Deactivated']

In [4459]:
def cosine_distance(emb_i, emb_j):
  return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

## Stage 1.1 Class name mapping

In [4460]:
# get name embedding
# all info
similarity_mde = []
mde_embedding = sgram_mde
threshold = 0.7
similarity_mde_dsl = []
percentage = 0.8
for index, node in enumerate(ref_cls.cls_name):
  cls = node.split()[-1].strip() # get the class name, remove abstract key word
  
  mde_emb_i = get_mde_embedding(cls, mde_embedding)
  mde_emb_i_dsl = get_mde_embedding(ref_cls.cls_atr[node]["dsl"], mde_embedding)

  pair = []
  mde_pair = []
  for j, stu_node in enumerate(stu_cls.cls_name):
    cls = stu_node.split()[-1].strip() # get the class name, remove abstract key word
    print()
    mde_emb_j = get_mde_embedding(cls, mde_embedding)
    mde_emb_j_dsl = get_mde_embedding(stu_cls.cls_atr[stu_node]["dsl"], mde_embedding)

    mde_sim = cosine_distance(mde_emb_i, mde_emb_j)
    mde_sim_dsl = cosine_distance(mde_emb_i_dsl, mde_emb_j_dsl)

    mde_pair.append(mde_sim * percentage + (1-percentage) * mde_sim_dsl)

  # apply third quartile
  similarity_mde.append(mde_pair)

dict_sim_word = {}
for i in range(len(ref_cls.cls_name)):
  dict_sim_word[ref_cls.cls_name[i]] = {}
  for j in range(len(stu_cls.cls_name)):
    dict_sim_word[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_mde[i][j]

In [4461]:
for key in dict_sim_word:
    exact_match = False
    exact_cls = ""
    for cls_2 in dict_sim_word[key]:
        if dict_sim_word[key][cls_2] > 0.99:
            exact_match = True
            exact_cls = cls_2
            print(cls_2)
            break
    
    if exact_match:    # keep the exact match, others to 0
        print("update")
        for cls_2 in dict_sim_word[key]:
            if cls_2 !=  exact_cls:
                dict_sim_word[key][cls_2] = 0

similarity_mde = []
for i in dict_sim_word:
    tmp = []
    for j in dict_sim_word[i]:
        tmp.append(dict_sim_word[i][j])

    similarity_mde.append(tmp)

RoomType
update
BillingInformation
update


In [4462]:
import pandas as pd

df = pd.DataFrame(index=reference_class, columns=stu_class, data=similarity_mde)
df

,RoomType,BookingStatus,PaymentType,HotelRating,HotelBookingManagementSystem,BusinessTraveller,BillingInformation,TravelPreference,Hotel,SpecialOffer,RoomAvailability,RegularBooking,AccommodationDeal,abstract BookingStatus,PreliminaryBooking,FinalizedBooking,ConfirmedBooking,CancelledBooking,CreditCardInformation
RoomType,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BookingStatus,0.417276,0.950814,0.524146,0.434472,0.717348,0.431577,0.451960,0.474790,0.455107,0.311174,0.428766,0.729993,0.365752,0.929731,0.741056,0.680430,0.735933,0.725983,0.404945
PaymentType,0.403901,0.510761,0.945078,0.344794,0.499869,0.392599,0.517732,0.393012,0.336580,0.403015,0.350155,0.463458,0.390009,0.517321,0.444215,0.450104,0.501350,0.517498,0.572045
HotelBookingManager,0.411749,0.700174,0.477571,0.616955,0.869835,0.470396,0.455592,0.469642,0.773188,0.317623,0.460285,0.707691,0.416947,0.724261,0.706409,0.648066,0.687924,0.677351,0.382796
HotelChain,0.311024,0.354715,0.341594,0.663262,0.693455,0.397784,0.332584,0.355941,0.804447,0.294366,0.375822,0.402391,0.382943,0.357943,0.434351,0.387463,0.367899,0.397377,0.291746
Traveller,0.230137,0.392869,0.304445,0.396521,0.474476,0.810079,0.431873,0.455774,0.420221,0.341422,0.315120,0.378396,0.352967,0.394879,0.452741,0.369274,0.402113,0.402763,0.343432
BillingInformation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
TravelPreference,0.280731,0.451204,0.379848,0.450227,0.543773,0.528986,0.407387,0.935521,0.431168,0.367817,0.368790,0.454925,0.444117,0.453402,0.518085,0.412377,0.450702,0.455096,0.351299
Hotel,0.373558,0.427193,0.308634,0.732863,0.804965,0.390898,0.405672,0.409515,0.973505,0.326743,0.456470,0.477488,0.438255,0.447216,0.553960,0.419551,0.448087,0.445587,0.340434
abstract Offer,0.191393,0.240495,0.318664,0.218947,0.301632,0.277345,0.276246,0.258333,0.249774,0.669884,0.282248,0.215396,0.293032,0.268474,0.204587,0.171866,0.276751,0.286296,0.207283


### Stage 1.1.1 Mapping (enum / regular + abstract)

In [4463]:
ref_enum = []
ref_reg_cls = []
stu_enum = []
stu_reg_cls = []

for enum in ref_cls.cls_name:
  if ref_cls.cls_atr[enum]["type"] == "enum":
    ref_enum.append(enum)
  else:
    ref_reg_cls.append(enum)

for enum in stu_cls.cls_name:
  if stu_cls.cls_atr[enum]["type"] == "enum":
    stu_enum.append(enum)
  else:
    stu_reg_cls.append(enum)

In [4464]:
enum_mapping = match_classes(ref_enum, stu_enum, dict_sim_word, thresh = 0.5)
enum_mapping[0]

{'RoomType': {'RoomType': 0.9999999999999999, 'BookingStatus': 0, 'PaymentType': 0, 'HotelRating': 0, 'HotelBookingManagementSystem': 0, 'BusinessTraveller': 0, 'BillingInformation': 0, 'TravelPreference': 0, 'Hotel': 0, 'SpecialOffer': 0, 'RoomAvailability': 0, 'RegularBooking': 0, 'AccommodationDeal': 0, 'abstract BookingStatus': 0, 'PreliminaryBooking': 0, 'FinalizedBooking': 0, 'ConfirmedBooking': 0, 'CancelledBooking': 0, 'CreditCardInformation': 0}, 'BookingStatus': {'RoomType': 0.4172764538666226, 'BookingStatus': 0.9508141644108056, 'PaymentType': 0.5241455868344695, 'HotelRating': 0.4344716732215805, 'HotelBookingManagementSystem': 0.7173481002881363, 'BusinessTraveller': 0.43157678852641745, 'BillingInformation': 0.4519601960602197, 'TravelPreference': 0.47478994627586624, 'Hotel': 0.45510740827348384, 'SpecialOffer': 0.31117415882730476, 'RoomAvailability': 0.42876579936188375, 'RegularBooking': 0.7299933713778834, 'AccommodationDeal': 0.3657524818001095, 'abstract BookingSt

[('RoomType', 'RoomType'),
 ('BookingStatus', 'BookingStatus'),
 ('PaymentType', 'PaymentType'),
 (None, 'HotelRating')]

In [4465]:
cls_mapping = match_classes(ref_reg_cls, stu_reg_cls, dict_sim_word, thresh = 0.5)
cls_mapping[0]

{'RoomType': {'RoomType': 0.9999999999999999, 'BookingStatus': 0, 'PaymentType': 0, 'HotelRating': 0, 'HotelBookingManagementSystem': 0, 'BusinessTraveller': 0, 'BillingInformation': 0, 'TravelPreference': 0, 'Hotel': 0, 'SpecialOffer': 0, 'RoomAvailability': 0, 'RegularBooking': 0, 'AccommodationDeal': 0, 'abstract BookingStatus': 0, 'PreliminaryBooking': 0, 'FinalizedBooking': 0, 'ConfirmedBooking': 0, 'CancelledBooking': 0, 'CreditCardInformation': 0}, 'BookingStatus': {'RoomType': 0.4172764538666226, 'BookingStatus': 0.9508141644108056, 'PaymentType': 0.5241455868344695, 'HotelRating': 0.4344716732215805, 'HotelBookingManagementSystem': 0.7173481002881363, 'BusinessTraveller': 0.43157678852641745, 'BillingInformation': 0.4519601960602197, 'TravelPreference': 0.47478994627586624, 'Hotel': 0.45510740827348384, 'SpecialOffer': 0.31117415882730476, 'RoomAvailability': 0.42876579936188375, 'RegularBooking': 0.7299933713778834, 'AccommodationDeal': 0.3657524818001095, 'abstract BookingSt

[('HotelBookingManager', 'RegularBooking'),
 ('HotelChain', 'HotelBookingManagementSystem'),
 ('Traveller', 'BusinessTraveller'),
 ('BillingInformation', 'BillingInformation'),
 ('TravelPreference', 'PreliminaryBooking'),
 ('Hotel', 'Hotel'),
 ('abstract Offer', None),
 ('SpecialOffer', 'SpecialOffer'),
 ('RegularOffer', None),
 ('TravelSearch', 'TravelPreference'),
 ('SearchInfo', None),
 ('RoomAvailability', 'RoomAvailability'),
 ('abstract TripInfo', None),
 ('BookingInfo', 'abstract BookingStatus'),
 ('Booking', 'ConfirmedBooking'),
 (None, 'AccommodationDeal'),
 (None, 'FinalizedBooking'),
 (None, 'CancelledBooking'),
 (None, 'CreditCardInformation')]

In [4466]:
mapping = enum_mapping[0] + cls_mapping[0]

for map in mapping:
  if map[0] and map[1]:
    ref_cls.cls_atr[map[0]]['score'] = 1
    ref_cls.cls_atr[map[0]]['counterpart'] = map[1]

    stu_cls.cls_atr[map[1]]['score'] = 1
    stu_cls.cls_atr[map[1]]['counterpart'] = map[0]

    if ref_cls.cls_atr[map[0]]["type"] !=  stu_cls.cls_atr[map[1]]["type"]:
      # type mismatch
      ref_cls.cls_atr[map[0]]['score'] = 0.5
      stu_cls.cls_atr[map[1]]["score"] = 0.5

In [4467]:
ref_cls.cls_atr

{'RoomType': {'score': 1,
  'type': 'enum',
  'dsl': 'RoomType(SINGLE, DOUBLE, TWIN)',
  'counterpart': 'RoomType',
  'attributes': {'SINGLE': {'score': 0, 'counterpart': None},
   'DOUBLE': {'score': 0, 'counterpart': None},
   'TWIN': {'score': 0, 'counterpart': None}}},
 'BookingStatus': {'score': 1,
  'type': 'enum',
  'dsl': 'BookingStatus(FINALIZED, CONFIRMED, CANCELLED_BY_HBMS, PRELIMINARY, CANCELLED_BY_TRAVELLER, CANCELLED_BY_HOTEL)',
  'counterpart': 'BookingStatus',
  'attributes': {'FINALIZED': {'score': 0, 'counterpart': None},
   'CONFIRMED': {'score': 0, 'counterpart': None},
   'CANCELLED_BY_HBMS': {'score': 0, 'counterpart': None},
   'PRELIMINARY': {'score': 0, 'counterpart': None},
   'CANCELLED_BY_TRAVELLER': {'score': 0, 'counterpart': None},
   'CANCELLED_BY_HOTEL': {'score': 0, 'counterpart': None}}},
 'PaymentType': {'score': 1,
  'type': 'enum',
  'dsl': 'PaymentType (PREPAID, PAID_AT_HOTEL)',
  'counterpart': 'PaymentType',
  'attributes': {'PREPAID': {'score':

## Stage 1.2 Class Mapping (with all info)

In [4468]:
ref_classes = []
stu_classes = []

for cls in ref_cls.cls_name:
  if ref_cls.cls_atr[cls]["counterpart"] == None:
    ref_classes.append(cls)

for cls in stu_cls.cls_name:
  if stu_cls.cls_atr[cls]["counterpart"] == None:
    stu_classes.append(cls)

In [4469]:
# get name embedding
# all info
similarity_all = []
mde_embedding = sgram_mde
threshold = 0.7




# reuse saved embedding result...
# may cause problem
# if os.path.exists(student_dir + "ada_embedding.txt"):
#   pass
  
  # with open(student_dir + "ada_embedding.txt", 'r') as json_file:
  #   dict_sim_all = json.load(json_file)

if False:
  pass

else:
  for index, node in enumerate(ref_cls.cls_name):
    text_1 = get_all_info(index, ref_cls.cls_name, ref_cls.raw_dsl, ref_cls.rel.raw_dsl, True)
    emb_i = get_embedding(text_1)
    mde_pair = []
    for j, stu_node in enumerate(stu_cls.cls_name):
      text_2 = get_all_info(j, stu_cls.cls_name, stu_cls.raw_dsl, stu_cls.rel.raw_dsl, True)
      emb_j = get_embedding(text_2)
      mde_sim = cosine_distance(emb_i, emb_j)
      mde_pair.append(mde_sim)

    # apply third quartile
    similarity_all.append(mde_pair)

  dict_sim_all = {}
  for i in range(len(ref_cls.cls_name)):
    dict_sim_all[ref_cls.cls_name[i]] = {}
    for j in range(len(stu_cls.cls_name)):
      dict_sim_all[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_all[i][j]





In [4470]:
with open(student_dir + "ada_embedding.txt", 'w') as json_file:
    json.dump(dict_sim_all, json_file)

In [4471]:
student_dir

'results/gpt-4/yujing/08_HBMS/'

In [4472]:
ref_classes

['abstract Offer', 'RegularOffer', 'SearchInfo', 'abstract TripInfo']

In [4473]:
stu_classes

['HotelRating',
 'AccommodationDeal',
 'FinalizedBooking',
 'CancelledBooking',
 'CreditCardInformation']

In [4474]:
dict_sim_all

{'RoomType': {'RoomType': 0.9909474228763404,
  'BookingStatus': 0.803173552380923,
  'PaymentType': 0.8356707840767582,
  'HotelRating': 0.8656606555287104,
  'HotelBookingManagementSystem': 0.8064718432994785,
  'BusinessTraveller': 0.7864257585806163,
  'BillingInformation': 0.7348513553502879,
  'TravelPreference': 0.8377312087593446,
  'Hotel': 0.8208074443952517,
  'SpecialOffer': 0.8013737019585344,
  'RoomAvailability': 0.8344231884840786,
  'RegularBooking': 0.7748748241275621,
  'AccommodationDeal': 0.8101307113313594,
  'abstract BookingStatus': 0.7869862865542151,
  'PreliminaryBooking': 0.7854488525915789,
  'FinalizedBooking': 0.7426128517704281,
  'ConfirmedBooking': 0.752288714795395,
  'CancelledBooking': 0.7227882411866817,
  'CreditCardInformation': 0.7490899683486841},
 'BookingStatus': {'RoomType': 0.8027120785275237,
  'BookingStatus': 0.9434973030104289,
  'PaymentType': 0.843292947147087,
  'HotelRating': 0.807593369010461,
  'HotelBookingManagementSystem': 0.81

In [4475]:
mapping = match_classes(ref_classes, stu_classes, dict_sim_all, 0.8)
mapping[0]

{'RoomType': {'RoomType': 0.9909474228763404, 'BookingStatus': 0.803173552380923, 'PaymentType': 0.8356707840767582, 'HotelRating': 0.8656606555287104, 'HotelBookingManagementSystem': 0.8064718432994785, 'BusinessTraveller': 0.7864257585806163, 'BillingInformation': 0.7348513553502879, 'TravelPreference': 0.8377312087593446, 'Hotel': 0.8208074443952517, 'SpecialOffer': 0.8013737019585344, 'RoomAvailability': 0.8344231884840786, 'RegularBooking': 0.7748748241275621, 'AccommodationDeal': 0.8101307113313594, 'abstract BookingStatus': 0.7869862865542151, 'PreliminaryBooking': 0.7854488525915789, 'FinalizedBooking': 0.7426128517704281, 'ConfirmedBooking': 0.752288714795395, 'CancelledBooking': 0.7227882411866817, 'CreditCardInformation': 0.7490899683486841}, 'BookingStatus': {'RoomType': 0.8027120785275237, 'BookingStatus': 0.9434973030104289, 'PaymentType': 0.843292947147087, 'HotelRating': 0.807593369010461, 'HotelBookingManagementSystem': 0.8191117182644779, 'BusinessTraveller': 0.800528

[('abstract Offer', None),
 ('RegularOffer', None),
 ('SearchInfo', None),
 ('abstract TripInfo', 'AccommodationDeal'),
 (None, 'HotelRating'),
 (None, 'FinalizedBooking'),
 (None, 'CancelledBooking'),
 (None, 'CreditCardInformation')]

In [4476]:
for map in mapping[0]:
  if map[0] and map[1]:
    ref_cls.cls_atr[map[0]]['score'] = 1
    ref_cls.cls_atr[map[0]]['counterpart'] = map[1]

    stu_cls.cls_atr[map[1]]['score'] = 1
    stu_cls.cls_atr[map[1]]['counterpart'] = map[0]

    if ref_cls.cls_atr[map[0]]["type"] !=  stu_cls.cls_atr[map[1]]["type"]:
      # type mismatch
      ref_cls.cls_atr[map[0]]['score'] = 0.5
      stu_cls.cls_atr[map[1]]["score"] = 0.5

## Stage 2.1 Attribute mapping

In [4477]:
def get_cosine_disntance(text1, text2, embedding):
    avg = np.sum(embedding[text1], axis=0)
    emb_i = avg / len(text1)

    avg = np.sum(embedding[text2], axis=0)
    emb_j = avg / len(text2)

    return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

In [4478]:
def create_cosine_similarity_dict(attr_1, attr_2, embedding):
  # attr_1: list[list] split camel case into separated case
  # attr_2: list[list] split camel case into separated case
  # embedding:
  # raw_1: list[str] camcel case attributes
  # raw_2: list[str] camcel case attributes

  # create cosine distance between two attributes with embeeding

  # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
  similarities = []
  for att in attr_1:
    emb_i = get_mde_embedding(att, embedding)
    pair = []
    for attribute in attr_2:
      emb_j = get_mde_embedding(attribute, embedding)

      sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
      pair.append(sim)

    similarities.append(pair)

  dict_attr = {}
  for i in range(len(attr_1)):
    dict_attr[attr_1[i]] = {}
    for j in range(len(attr_2)):
      dict_attr[attr_1[i]][attr_2[j]] = similarities[i][j]

  return dict_attr

In [4479]:
def match_attributes(raw_1, raw_2, dict_attr, threshold = 0.5):
  # map attributes lists raw_1 and raw_2
  # >>> mapAttributes(['deviceStatus'], ['id'], dic)[0]
  def node_subst_cost_attr(node1, node2):
    # threshod as 0.45
    if dict_attr[node1["name"]][node2["name"]] < threshold:
      return 3
    else:
      return 1-dict_attr[node1["name"]][node2["name"]]

  G_att_1 = nx.Graph()
  G_att_2 = nx.Graph()
  for node in raw_1:
    G_att_1.add_node(node, name = node)

  for node in raw_2:
    G_att_2.add_node(node, name = node)

  for v in nx.optimize_edit_paths(G_att_1, G_att_2, node_subst_cost=node_subst_cost_attr, edge_match=None, timeout = 20):
      minv = v
  # minv

  return minv

In [4480]:
def check_attributes_type(attr_1, attr_2, ref_dict, stu_dict):
  # given two attribute, compare whether their type can match
  attr_1 = attr_1.split() # from reference solution do not need to check it
  attr_2 = attr_2.split()

  # if enumerate literal and attributes
  # if len(attr_1) != len(attr_2):
  #   return 0.5, 0.5

  # if an attributes use a regular class as its type
  if len(attr_2) == 2:
    t = attr_2[0].strip()
    cls = stu_dict.get(t, None)
    if cls is not None and (stu_dict[t]['type'] == "regular" or stu_dict[t]['type'] == "abstract"):
      return 0.5, 0.5
    
  if len(attr_1) == 2 or len(attr_2) == 2:
    t = attr_1[0].strip()
    cls = ref_dict.get(t, None)
    # print(cls)
    t_2 = attr_2[0].strip()
    cls_2 = stu_dict.get(t_2, None)   
    # print(cls_2) 
    if cls is not None:
      if ref_dict[t]['counterpart'] != t_2:
        return 0.5, 0.5
    if cls_2 is not None:
      if stu_dict[t_2]['counterpart'] != t:
        return 0.5, 0.5
    
  return 1,1

In [4481]:
x =check_attributes_type(('CommandStatus commandStatus', 'ControllerStatus status')[0], ('CommandStatus commandStatus', 'ControllerStatus status')[1], ref_cls.cls_atr, stu_cls.cls_atr)
x

(1, 1)

### Stage 2.1.1 Attribute <-> Attribute (mapped classes)

In [4482]:
# map classes
pairs = []

# get matched pairs
for key in ref_cls.cls_atr:
  if ref_cls.cls_atr[key]["counterpart"] != None:
    pair = [key, ref_cls.cls_atr[key]["counterpart"]]
    pairs.append(pair)

for pair in pairs:
  if pair[0] and pair[1]:
    print(pair)

    # map attributes:
    # matched_ref = set()
    # matched_attr = set()

    raw_1 = []
    for attributes in ref_cls.cls_atr[pair[0]]["attributes"]:
      raw_1.append(attributes)

    raw_2 = []
    for attributes in stu_cls.cls_atr[pair[1]]["attributes"]:
      raw_2.append(attributes)

    cos_dict = create_cosine_similarity_dict(raw_1, raw_2, sgram_mde)
    mappings = match_attributes(raw_1, raw_2, cos_dict)[0]
    print("mapping", mappings)

    for mapping in mappings:
      if mapping[0] != None and mapping[1] != None:
        scores = check_attributes_type(mapping[0], mapping[1], ref_cls.cls_atr, stu_cls.cls_atr)
        ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]['score'] = scores[0]
        ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]["counterpart"] = (mapping[1], pair[1])
        stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]['score'] = scores[1]
        stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]["counterpart"] =  (mapping[0], pair[0])

    print("="*20)

['RoomType', 'RoomType']
mapping [('SINGLE', 'Single'), ('DOUBLE', 'Double'), ('TWIN', 'Twin')]
['BookingStatus', 'BookingStatus']
mapping [('FINALIZED', 'Finalized'), ('CONFIRMED', 'Confirmed'), ('CANCELLED_BY_HBMS', None), ('PRELIMINARY', 'Preliminary'), ('CANCELLED_BY_TRAVELLER', None), ('CANCELLED_BY_HOTEL', None), (None, 'Regular'), (None, 'Cancelled')]
['PaymentType', 'PaymentType']
mapping [('PREPAID', None), ('PAID_AT_HOTEL', 'PaidAtHotel'), (None, 'PrePaid')]
['HotelBookingManager', 'RegularBooking']
mapping [(None, 'string bookingIdentifier')]
['HotelChain', 'HotelBookingManagementSystem']
mapping [(None, 'string city'), (None, 'date dateOfArrival'), (None, 'date dateOfDeparture'), (None, 'int numberOfRooms'), (None, 'string typeOfRooms'), (None, 'int hotelRating'), (None, 'float budget'), (None, 'string travelPeriod'), (None, 'string bookingStatus'), (None, 'string bookingIdentifier')]
['Traveller', 'BusinessTraveller']
mapping [('string name', 'string name'), ('int reliabil

### Stage 2.1.2 Attribute <-> Attribute (between any classes)

In [4483]:
def create_cosine_similarity_list(attr_1, attr_2, embedding):
  # attr_1: list[list] split camel case into separated case
  # attr_2: list[list] split camel case into separated case
  # embedding:
  # raw_1: list[str] camcel case attributes
  # raw_2: list[str] camcel case attributes

  # create cosine distance between two attributes with embeeding

  # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
  similarities = []
  for att in attr_1:
    emb_i = get_mde_embedding(att, embedding)
    pair = []
    for attribute in attr_2:
      emb_j = get_mde_embedding(attribute, embedding)

      sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
      pair.append(sim)

    similarities.append(pair)

  return similarities

In [4484]:
def combine_two_dict(list_1, list_2, atr_1, atr_2, cls_1, cls_2, weight_1 = 0.9, weight_2=0.1):
  # list_1: similarity between cls
  # list_2: cos similarity between attributes
  if weight_1 + weight_2 != 1:
    raise ValueError("weight_1 + weight_2 != 1")
  result = {}
  for i in range(len(atr_1)):
    atr_r = atr_1[i]
    cls_r = cls_1[i]
    tmp_ref = (atr_r, cls_r)
    result[tmp_ref] = {}
    for j in range(len(atr_2)):
      atr_s = atr_2[j]
      cls_s = cls_2[j]
      tmp_stu = (atr_s, cls_s)

      result[tmp_ref][tmp_stu] = weight_1 * list_1[i][j] + weight_2 * list_2[i][j]

  return result


In [4485]:
# attr between any classes

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
  for attributes in ref_cls.cls_atr[cls]["attributes"]:
    if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_1.append(attributes)
      ref_source.append(cls)
      tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get attributes on student sides
for cls in stu_cls.cls_atr:
  for attributes in stu_cls.cls_atr[cls]["attributes"]:
    if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_2.append(attributes)
      stu_source.append(cls)
      tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)


    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = min(scores[0], 0.5)
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = mapping[1]

    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = min(scores[1], 0.5)
    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] =  mapping[0]

    if ref_cls.cls_atr[mapping[0][1]]['counterpart'] == mapping[1][1]:
      ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = min(scores[0], 1)

      stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = min(scores[1], 1)
print("="*20)

(('CANCELLED_BY_HBMS', 'BookingStatus'), None)
(('CANCELLED_BY_TRAVELLER', 'BookingStatus'), None)
(('CANCELLED_BY_HOTEL', 'BookingStatus'), ('string hotelChain', 'Hotel'))
(('PREPAID', 'PaymentType'), ('CreditCardInformation creditCardInformation', 'FinalizedBooking'))
(('string creditCardNumber', 'Booking'), ('string cardNumber', 'CreditCardInformation'))
(('BookingStatus bookingStatus', 'Booking'), ('string bookingStatus', 'HotelBookingManagementSystem'))
(('PaymentType paymentType', 'Booking'), ('string cardHolderName', 'CreditCardInformation'))
(('date confirmationDate', 'Booking'), ('date dateOfArrival', 'HotelBookingManagementSystem'))
(('float paidAmount', 'Booking'), ('float budget', 'HotelBookingManagementSystem'))
(None, ('Regular', 'BookingStatus'))
(None, ('Cancelled', 'BookingStatus'))
(None, ('PrePaid', 'PaymentType'))
(None, ('OneStar', 'HotelRating'))
(None, ('TwoStar', 'HotelRating'))
(None, ('ThreeStar', 'HotelRating'))
(None, ('FourStar', 'HotelRating'))
(None, ('Fi

## Stage 2.2.1 Attribute mapping atr -> cls

In [4486]:
#  get non-mapped attributes on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
  for attributes in ref_cls.cls_atr[cls]["attributes"]:
    if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_1.append(attributes)
      ref_source.append(cls)
      tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
    if stu_cls.cls_atr[cls]["counterpart"] == None:
      raw_2.append(cls)
      stu_source.append(cls)
      tup_s.append((cls, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = 0.5
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = (None, mapping[1][1])

    stu_cls.cls_atr[mapping[1][1]]['score'] = 0.5
    stu_cls.cls_atr[mapping[1][1]]["counterpart"] =  mapping[0]

print("="*20)


(('CANCELLED_BY_HBMS', 'BookingStatus'), None)
(('CANCELLED_BY_TRAVELLER', 'BookingStatus'), None)
(None, ('HotelRating', 'HotelRating'))
(None, ('FinalizedBooking', 'FinalizedBooking'))
(None, ('CancelledBooking', 'CancelledBooking'))
(None, ('CreditCardInformation', 'CreditCardInformation'))


## Stage 2.2.2 Attribute mapping cls -> atr

In [4487]:
#  get non-mapped cls on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get class on instrucotr sides
for cls in ref_cls.cls_atr:
    if ref_cls.cls_atr[cls]["counterpart"] == None:
      raw_1.append(cls)
      ref_source.append(cls)
      tup_r.append((cls, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
  for attributes in stu_cls.cls_atr[cls]["attributes"]:
    if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_2.append(attributes)
      stu_source.append(cls)
      tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
    ref_cls.cls_atr[mapping[0][1]]['score'] = 0.5
    ref_cls.cls_atr[mapping[0][1]]["counterpart"] = mapping[1]

    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = 0.5
    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] = (None, mapping[0][1])

print("="*20)


(('abstract Offer', 'abstract Offer'), None)
(('RegularOffer', 'RegularOffer'), ('Regular', 'BookingStatus'))
(('SearchInfo', 'SearchInfo'), None)
(None, ('Cancelled', 'BookingStatus'))
(None, ('PrePaid', 'PaymentType'))
(None, ('OneStar', 'HotelRating'))
(None, ('TwoStar', 'HotelRating'))
(None, ('ThreeStar', 'HotelRating'))
(None, ('FourStar', 'HotelRating'))
(None, ('FiveStar', 'HotelRating'))
(None, ('string city', 'HotelBookingManagementSystem'))
(None, ('date dateOfDeparture', 'HotelBookingManagementSystem'))
(None, ('int numberOfRooms', 'HotelBookingManagementSystem'))
(None, ('string typeOfRooms', 'HotelBookingManagementSystem'))
(None, ('int hotelRating', 'HotelBookingManagementSystem'))
(None, ('string travelPeriod', 'HotelBookingManagementSystem'))
(None, ('string bookingIdentifier', 'HotelBookingManagementSystem'))
(None, ('BillingInformation billingInformation', 'BusinessTraveller'))
(None, ('TravelPreference travelPreference', 'BusinessTraveller'))
(None, ('boolean breakf

## Stage 3: Relationship mapping

In [4488]:
def check_relations_classes(ref_index, stu_index, ref_elements, stu_elements):
    # ref_index: index of the element in the dsl
    # stu_index: index of the element in the dsl
    # ref_elements: list[str] elements in the dsl
    # stu_elements: list[str] elements in the dsl

    ref_class_1 = ref_elements[ref_index]
    # check whether this is an abstract class
    dict_keys = list(ref_attributes.keys())
    if not ref_class_1 in dict_keys:
      abstract = "abstract " + ref_class_1
      ref_class_1 = abstract

    true_pair = ref_attributes[ref_class_1]["counterpart"]
    if isinstance(true_pair, tuple):
      boolean_1 = False
    elif true_pair == None:
      boolean_1 = False
    else:
      tmp = true_pair.replace("abstract ", "")
      boolean_1 = stu_elements[stu_index] == tmp
    return boolean_1

#### Compare rel

In [4489]:

def compare_edges(ref_e, stu_e):
  # compare if two relationships match
  # return bool, score, score

  ref_elements = ref_e.split()
  stu_elements = stu_e.split()
  # print(ref_elements, stu_elements)

  # exact match first
  # length must match
  n_ref = len(ref_elements)
  n_stu = len(stu_elements)
  if n_ref == n_stu and n_ref == 5:
    # all element must match
    boolean_1 = check_relations_classes(1,1,ref_elements,stu_elements )

    boolean_4 = check_relations_classes(4,4,ref_elements,stu_elements )

    # type
    boolean_2 = ref_elements[2] == stu_elements[2]

    # association
    boolean_0 = ref_elements[0] == stu_elements[0]

    # association
    boolean_3 = ref_elements[3] == stu_elements[3]

    if boolean_0 and boolean_1 and boolean_2 and boolean_3 and boolean_4:
      print("exact match success")
      print(ref_e, "|||", stu_e)
      return True, 1, 1

    boolean_1_4 = False
    boolean_4_1 = False
    boolean_1_4 = check_relations_classes(1,4,ref_elements,stu_elements )
    boolean_4_1 = check_relations_classes(4,1,ref_elements,stu_elements )

    # check if the relationship is fliped, this only works for associate, not containment
    if  "associate" in ref_elements[2] and "associate" in stu_elements[2]:
      # association
      boolean_0 = ref_elements[0] == stu_elements[3]

      # association
      boolean_3 = ref_elements[3] == stu_elements[0]

    if boolean_0 and boolean_1_4 and boolean_3 and boolean_4_1 and boolean_2:
      print("match success, flipped associate")
      print(ref_e, "|||", stu_e)
      return True, 1, 1

    # check if relationship is partially match
    if  (boolean_1 and boolean_4) or (boolean_1_4 and boolean_4_1):
      print("match partially success")
      print(ref_e, "|||", stu_e)
      return True, 0.5, 0.5

  if n_ref == n_stu and n_ref == 3:
    # all element must match
    boolean_0 = check_relations_classes(0,0,ref_elements,stu_elements )
    boolean_2 = check_relations_classes(2,2,ref_elements,stu_elements )

    # type
    boolean_1 = ref_elements[1] == stu_elements[1]

    if boolean_0 and boolean_1 and boolean_2:
      print("match success")
      print(ref_e, "|||", stu_e)
      # ref_dict[ref_e]["score"] = 1
      # ref_dict[ref_e]["counterpart"] = stu_e

      # stu_dict[stu_e]["score"] = 1
      # stu_dict[stu_e]["counterpart"] = ref_e
      return True, 1, 1


  return False,0, 0

In [4490]:

for i, ref_tmp in enumerate(edges[0].raw_dsl):
  matchings = []
  for j, stu_tmp in enumerate(edges[1].raw_dsl):
    if edges[0].rels[i]["counterpart"] is None and edges[1].rels[j]["counterpart"] is None:
      result = compare_edges(ref_tmp, stu_tmp)
      if result[0]:
        matching = (i, j, result[1], result[2])
        matchings.append(matching)
        # matched_stu_edges.add(j)
        # matched_ref_edges.add(i)
        # edges[0].rels[i]["score"] = result[1]
        # edges[0].rels[i]["counterpart"] = stu_tmp

        # edges[1].rels[j]["score"] = result[2]
        # edges[1].rels[j]["counterpart"] = ref_tmp
  print(matchings)
  if len(matchings) > 0:
    print()
    highest_score = 0
    matching = None

    for m in matchings:
      if m[2] > highest_score:
        matching = m
        highest_score = m[2]
    j_j = matching[1]
    i_i = matching[0]
    
    edges[0].rels[i_i]["score"] = m[2]
    edges[0].rels[i_i]["counterpart"] = edges[1].raw_dsl[j_j]

    edges[1].rels[j_j]["score"] = m[3]
    edges[1].rels[j_j]["counterpart"] = edges[0].raw_dsl[i_i]



## Result: Precision/Recall/F1

### save matching to disk

In [4491]:
if not os.path.exists(student_out_dir):
    os.makedirs(student_out_dir)

In [4492]:
# with open( student_out_dir + 'matching.pkl', 'wb') as outp:
#     pickle.dump(grader, outp, pickle.HIGHEST_PROTOCOL)

In [4493]:
student_out_dir

'results/gpt-4/yujing/08_HBMS/result/'

In [4494]:


# as requested in comment

with open(student_out_dir + 'solution_matching.txt', 'w') as file:
     file.write(json.dumps(grader.ref.cls_atr))


with open(student_out_dir + 'student_matching.txt', 'w') as file:
     file.write(json.dumps(grader.stu.cls_atr))

with open(student_out_dir + 'solution_matching_relationship.txt', 'w') as file:
     file.write(json.dumps(grader.ref.rel.rels))


with open(student_out_dir + 'student_matching_relationship.txt', 'w') as file:
     file.write(json.dumps(grader.stu.rel.rels))

In [4495]:
student_out_dir

'results/gpt-4/yujing/08_HBMS/result/'

###### to local

In [4496]:
# # # be careful...

# with open(student_dir + "human_eval/" + 'ref_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.ref.cls_atr))


# with open(student_dir + "human_eval/" + 'stu_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.stu.cls_atr))

# with open(student_dir + "human_eval/" + 'ref_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.ref.rel.rels))


# with open(student_dir + "human_eval/" + 'stu_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.stu.rel.rels))

In [4497]:
algo_result = {}
algo_result["class"] = {"precision": 0, "recall": 0, "f1":0}
algo_result["attribute"] = {"precision": 0, "recall": 0, "f1":0}
algo_result["relationship"] = {"precision": 0, "recall": 0, "f1":0}

### Class

In [4498]:
# recell
count = 0
recall = 0
for key in list(ref_attributes.keys()):
  count += 1
  recall += ref_attributes[key]["score"]
print(count, recall)

if count != 0:
  algo_result["class"]['recall'] = recall / count

18 14.5


In [4499]:
# precision
count = 0
precision = 0
for key in list(stu_attributes.keys()):
  count += 1
  precision += stu_attributes[key]["score"]

print(count, precision)
if count != 0:
  algo_result["class"]['precision'] = precision / count

if algo_result["class"]['recall']   + algo_result["class"]['precision'] != 0:
  algo_result["class"]['f1'] = 2* (algo_result["class"]['recall']   * algo_result["class"]['precision'] ) / ( algo_result["class"]['recall']   + algo_result["class"]['precision']  )


19 14.0


### Attribute

In [4500]:
# recall
count = 0
recall = 0
for key in list(ref_attributes.keys()):
  attrs = ref_attributes[key]["attributes"]

  for att in attrs:
    count += 1
    recall += ref_attributes[key]["attributes"][att]["score"]
print(count, recall)

if count != 0:
  algo_result["attribute"]['recall'] = recall / count

32 26.0


In [4501]:
count = 0
precision = 0
for key in list(stu_attributes.keys()):
  attrs = stu_attributes[key]["attributes"]

  for att in attrs:
    count += 1
    precision += stu_attributes[key]["attributes"][att]["score"]
print(count, precision)

if count != 0:
  algo_result["attribute"]['precision'] = precision / count

r = algo_result["attribute"]['recall']
p = algo_result["attribute"]['precision']

if r + p != 0:
  algo_result["attribute"]['f1'] = 2* (r * p) / (r + p)


68 26.5


### Relationship

In [4502]:
count = 0
recall = 0
for tmp in edges[0].rels:
  attrs = tmp["score"]

  count += 1
  recall += attrs
print(count, recall)

if count != 0:
  algo_result["relationship"]['recall'] = recall / count

0 0


In [4503]:
count = 0
precision = 0
for tmp in edges[1].rels:
  attrs = tmp["score"]

  count += 1
  precision += attrs
print(count, precision)

if count!= 0:
  algo_result["relationship"]['precision'] = precision / count
r = algo_result["relationship"]['recall']
p = algo_result["relationship"]['precision']

if r + p != 0:
  algo_result["relationship"]['f1'] = 2* (r * p ) / ( r + p )


0 0


In [4504]:

# as requested in comment

with open(student_out_dir + 'algo_result.txt', 'w') as file:
     file.write(json.dumps(algo_result))